In [ ]:
import numpy as np
import pandas as pd
import gzip
import ast
import pyarrow
print(pyarrow.__version__)
print(pd.__version__)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVR
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import re
import scipy.sparse as sp
from sklearn.decomposition import TruncatedSVD, MiniBatchTruncatedSVD
from sklearn.mixture import GaussianMixture

22.0.0
2.3.3


In [24]:
rows = []
with gzip.open('data/bundle_data.json.gz', 'rt') as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        rows.append(ast.literal_eval(line))

bundle_data = pd.DataFrame(rows)
bundle_data.head()

,bundle_final_price,bundle_url,bundle_price,bundle_name,bundle_id,items,bundle_discount
0,$66.46,http://store.steampowered.com/bundle/450/?utm_...,$73.86,Dharker Studio 2015 Complete,450,"[{'genre': 'Adventure, Indie, RPG', 'item_id':...",10%
1,$20.15,http://store.steampowered.com/bundle/1473/?utm...,$25.87,"Naruto Shippuden Uncut Season 4, Vol. 3",1473,"[{'genre': '', 'item_id': '528360', 'discounte...",22%
2,$20.15,http://store.steampowered.com/bundle/1474/?utm...,$25.87,"Naruto Shippuden Uncut Season 4, Vol. 4",1474,"[{'genre': '', 'item_id': '528380', 'discounte...",22%
3,$15.17,http://store.steampowered.com/bundle/1437/?utm...,$25.35,Pixel Puzzles Ultimate: Starter Kit,1437,"[{'genre': 'Casual, Free to Play, Indie, Simul...",40%
4,$20.15,http://store.steampowered.com/bundle/1466/?utm...,$25.87,"Naruto Shippuden Uncut Season 4, Vol. 1",1466,"[{'genre': '', 'item_id': '527830', 'discounte...",22%


In [ ]:
rows = []
with gzip.open('data/australian_users_items.json.gz', 'rt') as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        rows.append(ast.literal_eval(line)) 

rev_df = pd.DataFrame(rows)

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [ ]:
rev_df.head()
rev_df.shape[1]

In [6]:
rows = []

with gzip.open('data/steam_reviews.json.gz', 'rt') as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        rows.append(ast.literal_eval(line))

df = pd.DataFrame(rows)

df.to_parquet('steam_reviews.parquet', engine='pyarrow', index=False)


ArrowKeyError: No type extension with name arrow.py_extension_type found

In [ ]:
df.dtypes
df.to_parquet('data/steam_reviews.parquet', engine='pyarrow', index=False)

In [ ]:
df = pd.read_parquet('steam_reviews.parquet')

In [16]:
df.shape[1]
df.head()

,username,hours,products,product_id,page_order,date,text,early_access,page,found_funny,compensation,user_id
0,Chaos Syren,0.1,41.0,725280,0,2017-12-17,This would not be acceptable as an entertainme...,False,1,NaN,NaN,NaN
1,₮ʜᴇ Wᴀʀᴛᴏɴ,51.1,769.0,328100,0,2017-12-27,looks like a facebook game,False,1,NaN,NaN,NaN
2,hello?<,14.6,2.0,328100,1,2017-10-16,Better than Minecraft,False,1,2.0,Product received for free,NaN
3,Cyderine916,5.0,64.0,35140,0,2018-01-04,I love and idolized Batman and this game is Ma...,False,1,NaN,NaN,NaN
4,DarklyThinking,16.6,577.0,35140,1,2018-01-04,Still worth playing in 2018.\nProbably my favo...,False,1,NaN,NaN,76561198007483075


In [ ]:
def mark_caps(word):
    # Mark words that are all uppercase and longer than 1 character
    if word.isupper() and len(word) > 1:
        return word.lower() + " <allcaps>"
    return word

def clean_text(text, mark_allcaps=True):
    if not isinstance(text, str):
        return ""

    # Normalize any sequence of ♥ to a token
    text = re.sub(r'[♥]+', ' <censored> ', text)

    # Replace repeated punctuation / emojis with tokens
    text = re.sub(r'!{2,}', ' <exclaim> ', text)   # !!! → <exclaim>
    text = re.sub(r'\?{2,}', ' <question> ', text) # ??? → <question>
    text = re.sub(r':\)', ' <smile> ', text)       # :) → <smile>

    # Remove URLs
    text = re.sub(r'http\S+|www\.\S+', '', text)

    # Optionally mark all-caps words
    if mark_allcaps:
        text = ' '.join(mark_caps(w) for w in text.split())

    # Lowercase everything (words that weren't all-caps are fine)
    text = text.lower()

    # Allow letters, digits, common sentiment punctuation, +-/ and the token brackets <>
    text = re.sub(r'[^a-z0-9<>\s!\?\.,:\+\-/]', ' ', text)

    # Collapse multiple spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text


df['cleaned_text'] = df['text'].apply(clean_text)
df['received_free'] = (df['compensation'] == 'Product received for free').astype(int)
df = df.drop(columns=['compensation'])
df['found_funny'] = df['found_funny'].fillna(0).astype(int)
df['found_funny_log'] = np.log1p(df['found_funny'])
df['hours'] = np.log1p(df['hours'])
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# Drop rows with missing values in critical columns (hours, products, date)
df = df.dropna(subset=['hours', 'products', 'date'])
df['user_id'] = df['user_id'].fillna('unknown_user')
df['product_id'] = df['product_id'].fillna('unknown_product')
# Extract Date Features
df['review_month'] = df['date'].dt.month
df['review_dow'] = df['date'].dt.dayofweek

user_codes = df['user_id'].astype('category').cat.codes
item_codes = df['product_id'].astype('category').cat.codes

df.to_parquet('steam_reviews.parquet', index=False)

ValueError: negative axis 0 index: -1

In [36]:
df = df.dropna(subset=['user_id', 'product_id'])
user_codes = df['user_id'].astype('category').cat.codes
item_codes = df['product_id'].astype('category').cat.codes

R = sp.coo_matrix(
    (df['hours'], (user_codes, item_codes))
)

In [37]:
df.shape[0]

3158609

In [ ]:
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# # -------------------------
# # Text Features: TF-IDF
# # -------------------------
tfidf = TfidfVectorizer(max_features=1000, stop_words='english')
X_train_text = tfidf.fit_transform(train_df['cleaned_text'])
X_val_text = tfidf.transform(val_df['cleaned_text'])
X_test_text = tfidf.transform(test_df['cleaned_text'])

y_train = train_df['hours'].values
y_val = val_df['hours'].values

# -------------------------
# Numeric & Categorical Features
# -------------------------
numeric_features = ['products', 'found_funny', 'review_month', 'review_dow', 'received_free']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features)
    ]
)

X_train_meta = preprocessor.fit_transform(train_df)
X_val_meta = preprocessor.transform(val_df)
X_test_meta = preprocessor.transform(test_df)

# -------------------------
# Combine text + meta
# -------------------------
X_train = sp.hstack([X_train_text, X_train_meta])
X_val = sp.hstack([X_val_text, X_val_meta])
X_test = sp.hstack([X_test_text, X_test_meta])

# -------------------------
# Train LinearSVR
# -------------------------
svr = LinearSVR(max_iter=5000, random_state=42)
svr.fit(X_train, y_train)

y_val_pred = svr.predict(X_val)
y_test_pred = svr.predict(X_test)

# -------------------------
# Collaborative Filtering / Matrix Factorization
# -------------------------
# Create sparse user-item matrix
user_codes = df['user_id'].astype('category').cat.codes
item_codes = df['product_id'].astype('category').cat.codes

R = sp.coo_matrix((df['hours'], (user_codes, item_codes)))

# Use TruncatedSVD (approximate MF)
n_factors = 50
svd = TruncatedSVD(n_components=n_factors, random_state=42)
user_factors = svd.fit_transform(R)
item_factors = svd.components_.T

# Predicted matrix: hours
R_pred = user_factors @ item_factors.T

# -------------------------
# 8. EM / Probabilistic modeling (Gaussian Mixture on residuals)
# -------------------------
residuals = y_train - svr.predict(X_train)
residuals = residuals.reshape(-1, 1)

gmm = GaussianMixture(n_components=3, random_state=42)
gmm.fit(residuals)

# Example: probability of a residual belonging to each cluster
probs = gmm.predict_proba(residuals)

print("Pipeline ready. Shapes:")
print("X_train:", X_train.shape)
print("Sparse user-item matrix R:", R.shape)
print("User factors:", user_factors.shape)
print("Item factors:", item_factors.shape)
print("GMM probabilities:", probs.shape)

In [ ]:
# -------------------------
# 1. Train/Validation/Test split
# -------------------------
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

y_train = train_df['hours'].values
y_val = val_df['hours'].values
y_test = test_df['hours'].values

# -------------------------
# 2. Text Features: TF-IDF
# -------------------------
tfidf = TfidfVectorizer(max_features=1000, stop_words='english')
X_train_text = tfidf.fit_transform(train_df['cleaned_text'])
X_val_text = tfidf.transform(val_df['cleaned_text'])
X_test_text = tfidf.transform(test_df['cleaned_text'])

# -------------------------
# 3. Numeric Features
# -------------------------
numeric_features = ['products', 'found_funny', 'review_month', 'review_dow', 'received_free']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features)
    ]
)

X_train_meta = preprocessor.fit_transform(train_df)
X_val_meta = preprocessor.transform(val_df)
X_test_meta = preprocessor.transform(test_df)

# -------------------------
# 4. Combine text + meta
# -------------------------
X_train = sp.hstack([X_train_text, X_train_meta])
X_val = sp.hstack([X_val_text, X_val_meta])
X_test = sp.hstack([X_test_text, X_test_meta])

# -------------------------
# 5. Train LinearSVR
# -------------------------
svr = LinearSVR(max_iter=5000, random_state=42)
svr.fit(X_train, y_train)

y_train_svr = svr.predict(X_train)
y_val_svr = svr.predict(X_val)
y_test_svr = svr.predict(X_test)

# -------------------------
# 6. Collaborative Filtering / Matrix Factorization
# -------------------------
user_codes = df['user_id'].astype('category').cat.codes
item_codes = df['product_id'].astype('category').cat.codes

R = sp.coo_matrix((df['hours'], (user_codes, item_codes)))

n_factors = 50
svd = TruncatedSVD(n_components=n_factors, random_state=42)
user_factors = svd.fit_transform(R)
item_factors = svd.components_.T

R_pred = user_factors @ item_factors.T

# Map MF predictions to train/val/test users/items
user_map = dict(zip(df['user_id'], user_codes))
item_map = dict(zip(df['product_id'], item_codes))

def get_mf_pred(df_subset):
    u_idx = df_subset['user_id'].map(user_map).values
    i_idx = df_subset['product_id'].map(item_map).values
    return R_pred[u_idx, i_idx]

y_train_mf = get_mf_pred(train_df)
y_val_mf = get_mf_pred(val_df)
y_test_mf = get_mf_pred(test_df)

# -------------------------
# 7. EM / Gaussian Mixture on residuals
# -------------------------
residuals = y_train - y_train_svr
residuals = residuals.reshape(-1, 1)

gmm = GaussianMixture(n_components=3, random_state=42)
gmm.fit(residuals)
cluster_means = gmm.means_.flatten()

# -------------------------
# 8. Combine SVR + MF + GMM residual adjustment
# -------------------------
def combined_prediction(svr_pred, mf_pred, gmm, cluster_means, alpha=0.5):
    # residuals are unknown at inference, approximate with zeros
    residuals = np.zeros_like(svr_pred).reshape(-1, 1)
    probs = gmm.predict_proba(residuals)
    expected_residual = np.dot(probs, cluster_means)
    combined = alpha * svr_pred + (1 - alpha) * mf_pred + expected_residual
    return combined

y_val_combined = combined_prediction(y_val_svr, y_val_mf, gmm, cluster_means, alpha=0.5)
y_test_combined = combined_prediction(y_test_svr, y_test_mf, gmm, cluster_means, alpha=0.5)

# -------------------------
# 9. Evaluation
# -------------------------
def evaluate(y_true, y_pred, name=""):
    from sklearn.metrics import mean_squared_error, r2_score
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    print(f"{name} RMSE: {rmse:.4f}, R2: {r2:.4f}")

evaluate(y_val, y_val_combined, "Validation")
evaluate(y_test, y_test_combined, "Test")

print("Pipeline ready. Shapes:")
print("X_train:", X_train.shape)
print("Sparse user-item matrix R:", R.shape)
print("User factors:", user_factors.shape)
print("Item factors:", item_factors.shape)


In [ ]:
# -------------------------
# 0. Text cleaning
# -------------------------
def mark_caps(word):
    if word.isupper() and len(word) > 1:
        return word.lower() + " <allcaps>"
    return word

def clean_text(text, mark_allcaps=True):
    if not isinstance(text, str):
        return ""
    text = re.sub(r'[♥]+', ' <censored> ', text)
    text = re.sub(r'!{2,}', ' <exclaim> ', text)
    text = re.sub(r'\?{2,}', ' <question> ', text)
    text = re.sub(r':\)', ' <smile> ', text)
    if mark_allcaps:
        text = ' '.join(mark_caps(w) for w in text.split())
    text = text.lower()
    text = re.sub(r'[^a-z0-9<>\s!\?\.,:\+\-/]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# -------------------------
# 1. Preprocess dataframe
# -------------------------
df['cleaned_text'] = df['text'].apply(clean_text)
df['received_free'] = (df['compensation'] == 'Product received for free').astype(int)
df['found_funny'] = df['found_funny'].fillna(0).astype(int)
df['hours'] = np.log1p(df['hours'])
df['date'] = pd.to_datetime(df['date'], errors='coerce')

df = df.drop(columns=['compensation'])
df = df.dropna(subset=['hours', 'products', 'date'])
df['user_id'] = df['user_id'].fillna('unknown_user')
df['product_id'] = df['product_id'].fillna('unknown_product')

df['review_month'] = df['date'].dt.month
df['review_dow'] = df['date'].dt.dayofweek

user_codes = df['user_id'].astype('category').cat.codes
item_codes = df['product_id'].astype('category').cat.codes

# -------------------------
# 2. Train/Validation/Test split
# -------------------------
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

y_train = train_df['hours'].values
y_val = val_df['hours'].values
y_test = test_df['hours'].values

# -------------------------
# 3. Text Features: TF-IDF
# -------------------------
tfidf = TfidfVectorizer(max_features=1000, stop_words='english')
X_train_text = tfidf.fit_transform(train_df['cleaned_text'])
X_val_text = tfidf.transform(val_df['cleaned_text'])
X_test_text = tfidf.transform(test_df['cleaned_text'])

# -------------------------
# 4. Numeric Features
# -------------------------
numeric_features = ['products', 'found_funny', 'review_month', 'review_dow', 'received_free']
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numeric_features)
])

X_train_meta = preprocessor.fit_transform(train_df)
X_val_meta = preprocessor.transform(val_df)
X_test_meta = preprocessor.transform(test_df)

# -------------------------
# 5. Combine text + meta
# -------------------------
X_train = sp.hstack([X_train_text, X_train_meta])
X_val = sp.hstack([X_val_text, X_val_meta])
X_test = sp.hstack([X_test_text, X_test_meta])

# -------------------------
# 6. Train LinearSVR
# -------------------------
svr = LinearSVR(max_iter=5000, random_state=42)
svr.fit(X_train, y_train)

y_train_svr = svr.predict(X_train)
y_val_svr = svr.predict(X_val)
y_test_svr = svr.predict(X_test)

# -------------------------
# 7. Collaborative Filtering / MF
# -------------------------
R = sp.coo_matrix((df['hours'], (user_codes, item_codes)))

# Use MiniBatchTruncatedSVD for speed on large sparse matrix
n_factors = 50
svd = TruncatedSVD(n_components=n_factors, random_state=42)
user_factors = svd.fit_transform(R)
item_factors = svd.components_.T
R_pred = user_factors @ item_factors.T

# Map MF predictions
user_map = dict(zip(df['user_id'], user_codes))
item_map = dict(zip(df['product_id'], item_codes))

def get_mf_pred(df_subset):
    u_idx = df_subset['user_id'].map(user_map).values
    i_idx = df_subset['product_id'].map(item_map).values
    return R_pred[u_idx, i_idx]

y_train_mf = get_mf_pred(train_df)
y_val_mf = get_mf_pred(val_df)
y_test_mf = get_mf_pred(test_df)

# -------------------------
# 8. EM / Gaussian Mixture on residuals
# -------------------------
residuals = y_train - y_train_svr
residuals = residuals.reshape(-1, 1)

gmm = GaussianMixture(n_components=3, random_state=42)
gmm.fit(residuals)
cluster_means = gmm.means_.flatten()

# -------------------------
# 9. Combine SVR + MF + GMM residuals
# -------------------------
def combined_prediction(svr_pred, mf_pred, gmm, cluster_means, alpha=0.5):
    residuals = np.zeros_like(svr_pred).reshape(-1, 1)
    probs = gmm.predict_proba(residuals)
    expected_residual = np.dot(probs, cluster_means)
    combined = alpha * svr_pred + (1 - alpha) * mf_pred + expected_residual
    return combined

y_val_combined = combined_prediction(y_val_svr, y_val_mf, gmm, cluster_means)
y_test_combined = combined_prediction(y_test_svr, y_test_mf, gmm, cluster_means)

# -------------------------
# 10. Evaluation
# -------------------------
def evaluate(y_true, y_pred, name=""):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    print(f"{name} RMSE: {rmse:.4f}, R2: {r2:.4f}")

evaluate(y_val, y_val_combined, "Validation")
evaluate(y_test, y_test_combined, "Test")

print("Pipeline ready.")
print("X_train shape:", X_train.shape)
print("Sparse user-item matrix R:", R.shape)
print("User factors:", user_factors.shape)
print("Item factors:", item_factors.shape)